In [1]:
import numpy as np
import pandas as pd

In [2]:
%%time
train_df = pd.read_csv("../../01_input/train.csv")
test_df = pd.read_csv("../../01_input/test.csv")

CPU times: user 11.4 s, sys: 392 ms, total: 11.8 s
Wall time: 11.8 s


In [3]:
submit_flg=1 #全件処理かどうかをコントロール
grid_flg=1 #GridSearchをするかコントロール
SEED=12345
sample_num=10000
fold_num=5

#train関連
train_drop_col=['ID_code','target']
train_label='target'

#test関連
test_drop_col=['ID_code']

In [4]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
x_test=test_df.drop(test_drop_col,axis=1)

特徴量作成コードを記載

In [5]:
x_train.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [6]:
df_x_train_negcount = (x_train < 0)
df_x_test_negcount = (x_test < 0)

In [7]:
df_x_train_negcount['NegCount']=df_x_train_negcount.sum(axis=1)
df_x_test_negcount['NegCount']=df_x_test_negcount.sum(axis=1)

In [8]:
df_x_train_negcount

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,NegCount
0,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,True,False,False,False,True,36
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,33
2,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,False,True,False,False,False,42
3,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,True,True,False,False,True,39
4,False,True,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,True,39
5,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,38
6,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,True,32
7,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,36
8,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,True,True,False,False,False,40
9,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,40


In [9]:
df_x_train_negcount['NegCount'].to_csv('../../02_feature/113_train_NegativeCount.csv',index=False,header=True)
df_x_test_negcount['NegCount'].to_csv('../../02_feature/113_test_NegativeCount.csv',index=False,header=True)